In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['FP', 'FPs']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('fluticasone propionate', 213),
 ('fluorescence polarization', 132),
 ('fusion peptide', 107),
 ('false positive', 85),
 ('fluorescence protein', 76),
 ('floor plate', 58),
 ('flavopiridol', 42),
 ('fusion protein', 33),
 ('family planning', 27),
 ('false positives', 25),
 ('field potential', 23),
 ('flurbiprofen', 19),
 ('flagellar pocket', 16),
 ('foot processes', 16),
 ('field potentials', 12),
 ('family physicians', 11),
 ('first progression', 10),
 ('fat pad', 10),
 ('fluid percussion', 9),
 ('fertility preservation', 9),
 ('prostaglandin f2α receptor', 9),
 ('cp fine', 9),
 ('follicular phase', 8),
 ('fabry perot', 8),
 ('formyl peptide', 8),
 ('foreperiod', 8),
 ('fiber protein', 8),
 ('fluoropyrimidine', 8),
 ('fat percentage', 7),
 ('fluorophosphonate', 7),
 ('f prostanoid', 6),
 ('fish peptide', 6),
 ('flavoprotein', 6),
 ('fp dipeptides', 6),
 ('fluticasone', 6),
 ('fabry pérot', 5),
 ('few polyhedra', 5),
 ('fenugreek powder', 5),
 ('fenpropimorph', 5),
 ('fowlpox', 5),
 

In [6]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

INFO: [2020-09-25 14:47:31] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.2/bio_ontology.pkl


In [7]:
result = [grounding_map, names, pos_labels]

In [8]:
result

[{'benzaldehyde lyase': 'MESH:C059416',
  'betaine aldehyde': 'CHEBI:CHEBI:15710',
  'bile salt activity lipase': 'HGNC:1848',
  'bioartificial liver': 'MESH:D019164',
  'blood alcohol levels': 'ungrounded',
  'breath alcohol levels': 'ungrounded',
  'british anti lewisite': 'CHEBI:CHEBI:64198',
  'brochoalveolar lavage': 'MESH:D018893',
  'bronchalveolar lavage': 'MESH:D018893',
  'bronchial alveolar lavage': 'MESH:D018893',
  'bronchial lavage': 'MESH:D018893',
  'bronchio alveolar lavage': 'MESH:D018893',
  'bronchiolar lavage': 'MESH:D018893',
  'broncho alveolar lavage': 'MESH:D018893',
  'bronchoalveolar': 'MESH:D018893',
  'bronchoalveolar fluid': 'MESH:D018893',
  'bronchoalveolar larvage': 'MESH:D018893',
  'bronchoalveolar lavage': 'MESH:D018893'},
 {'MESH:C059416': 'benzaldehyde lyase',
  'CHEBI:CHEBI:15710': 'betaine aldehyde',
  'HGNC:1848': 'CEL',
  'MESH:D019164': 'Liver, Artificial',
  'CHEBI:CHEBI:64198': 'dimercaprol',
  'MESH:D018893': 'Bronchoalveolar Lavage'},
 ['H

In [6]:
grounding_map, names, pos_labels = grounding_map, names, pos_labels = [{'cp fine': 'MESH:D052638',
  'f poae': 'ungrounded',
  'f prostanoid': 'CHEBI:CHEBI:26347',
  'f series prostanoid': 'CHEBI:CHEBI:26347',
  'fabry perot': 'ungrounded',
  'fabry pérot': 'ungrounded',
  'faecal protease': 'ungrounded',
  'faecalibacterium prausnitzii': 'MESH:D000070037',
  'false positive': 'MESH:D005189',
  'false positives': 'MESH:D005189',
  'family physicians': 'MESH:D010821',
  'family planning': 'MESH:D005193',
  'farnesyl phosphate': 'CHEBI:CHEBI:24018',
  'fast pathway': 'CHEBI:CHEBI:34922',
  'fat pad': 'MESH:D000273',
  'fat percentage': 'ungrounded',
  'fatty pancreas': 'ungrounded',
  'female protein': 'ungrounded',
  'fenpropimorph': 'CHEBI:CHEBI:50145',
  'fenugreek powder': 'ungrounded',
  'fermentation production': 'ungrounded',
  'fertility preservation': 'MESH:D059247',
  'fetal pancreas': 'ungrounded',
  'few polyhedra': 'ungrounded',
  'fgfb pacap': 'ungrounded',
  'fiber protein': 'MESH:D012596',
  'field potential': 'field_potential',
  'field potentials': 'field_potential',
  'filamentous processes': 'ungrounded',
  'filter paper': 'ungrounded',
  'fine particles': 'MESH:D052638',
  'fipronil': 'CHEBI:CHEBI:5063',
  'first progression': 'ungrounded',
  'fish peptide': 'ungrounded',
  'fixed point': 'ungrounded',
  'flagellar pocket': 'GO:GO:0020016',
  'flavonoids and phenolic acid': 'ungrounded',
  'flavopiridol': 'CHEBI:CHEBI:47344',
  'flavoprotein': 'CHEBI:CHEBI:5086',
  'floor plate': 'floor_plate',
  'flow probe': 'ungrounded',
  'flow proneness': 'ungrounded',
  'flowering period': 'ungrounded',
  'fluid percussion': 'fluid_percussion',
  'fluid pressure': 'ungrounded',
  'fluorescence polarization': 'MESH:D005454',
  'fluorescence protein': 'MESH:D008164',
  'fluorophosphonate': 'CHEBI:CHEBI:42699',
  'fluoropyrimidine': 'PUBCHEM:141643',
  'fluphenazine': 'CHEBI:CHEBI:5123',
  'flurbiprofen': 'CHEBI:CHEBI:5130',
  'fluticasone': 'CHEBI:CHEBI:5134',
  'fluticasone propionate': 'CHEBI:CHEBI:31441',
  'focused pulsed': 'ungrounded',
  'follicular phase': 'MESH:D005498',
  'foot processes': 'NCIT:C32623',
  'footpad': 'ungrounded',
  'foreperiod': 'ungrounded',
  'formyl peptide': 'MESH:D009240',
  'fowlpox': 'MESH:D005587',
  'fowlpox virus': 'MESH:D005587',
  'fp dipeptides': 'ungrounded',
  'fractional photothermolysis': 'ungrounded',
  'frailty phenotype': 'ungrounded',
  'from propolis': 'MESH:D011429',
  'frontal pole': 'ungrounded',
  'fronto parietal': 'ungrounded',
  'frontopolar cortex': 'ungrounded',
  'fructus psoraleae': 'ungrounded',
  'fucan polysaccharides': 'ungrounded',
  'fungiform papilla': 'ungrounded',
  'fusion peptide': 'MESH:D014760',
  'fusion peptides': 'MESH:D014760',
  'fusion positive': 'ungrounded',
  'fusion protein': 'MESH:D014760',
  'fusogenic peptides': 'MESH:D014760',
  'prostaglandin f': 'HGNC:9600',
  'prostaglandin f2α receptor': 'HGNC:9600'},
 {'MESH:D052638': 'Particulate Matter',
  'CHEBI:CHEBI:26347': 'prostanoid',
  'MESH:D000070037': 'Faecalibacterium prausnitzii',
  'MESH:D005189': 'False Positive Reactions',
  'MESH:D010821': 'Physicians, Family',
  'MESH:D005193': 'Family Planning Services',
  'CHEBI:CHEBI:24018': 'farnesyl phosphate',
  'CHEBI:CHEBI:34922': 'picloram',
  'MESH:D000273': 'Adipose Tissue',
  'CHEBI:CHEBI:50145': 'fenpropimorph',
  'MESH:D059247': 'Fertility Preservation',
  'MESH:D012596': 'Scleroproteins',
  'field_potential': 'field_potential',
  'CHEBI:CHEBI:5063': 'fipronil',
  'GO:GO:0020016': 'ciliary pocket',
  'CHEBI:CHEBI:47344': 'alvocidib',
  'CHEBI:CHEBI:5086': 'flavoprotein',
  'floor_plate': 'floor_plate',
  'fluid_percussion': 'fluid_percussion',
  'MESH:D005454': 'Fluorescence Polarization',
  'MESH:D008164': 'Luminescent Proteins',
  'CHEBI:CHEBI:42699': 'fluoridophosphate',
  'PUBCHEM:141643': '2-Fluoropyrimidine',
  'CHEBI:CHEBI:5123': 'fluphenazine',
  'CHEBI:CHEBI:5130': 'flurbiprofen',
  'CHEBI:CHEBI:5134': 'fluticasone',
  'CHEBI:CHEBI:31441': 'fluticasone propionate',
  'MESH:D005498': 'Follicular Phase',
  'NCIT:C32623': 'Foot Process',
  'MESH:D009240': 'N-Formylmethionine Leucyl-Phenylalanine',
  'MESH:D005587': 'Fowlpox virus',
  'MESH:D011429': 'Propolis',
  'MESH:D014760': 'Viral Fusion Proteins',
  'HGNC:9600': 'PTGFR'},
 ['CHEBI:CHEBI:47344',
  'CHEBI:CHEBI:5130',
  'MESH:D005189',
  'MESH:D005193',
  'MESH:D005454',
  'MESH:D008164',
  'MESH:D014760',
  'NCIT:C32623',
 'GO:GO:0020016']]

In [7]:
excluded_longforms = []

In [8]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [9]:
additional_entities = {'HGNC:1848': ['CEL', ['CEL', 'BSSL', 'MODY8', 'carboxyl ester']]}

In [10]:
unambiguous_agent_texts = {}

In [11]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [12]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [13]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [14]:
intersection1

[('HGNC:1848', 'HGNC:1848', 86)]

In [15]:
intersection2

[('MESH:D018893', 'HGNC:1848', 0),
 ('HGNC:1848', 'HGNC:1848', 1),
 ('ungrounded', 'HGNC:1848', 0),
 ('CHEBI:CHEBI:64198', 'HGNC:1848', 0),
 ('MESH:D019164', 'HGNC:1848', 0),
 ('MESH:C059416', 'HGNC:1848', 0),
 ('CHEBI:CHEBI:15710', 'HGNC:1848', 0)]

In [17]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])

In [18]:
names.update({key: value[0] for key, value in additional_entities.values()})
pos_labels = list(set(pos_labels) | additional_entities.keys())

In [19]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-27 03:30:01] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-27 03:30:22] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Best f1 score of 0.9930679234710235 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [20]:
classifier.stats

{'label_distribution': {'MESH:D018893': 1259,
  'HGNC:1848': 86,
  'ungrounded': 17,
  'CHEBI:CHEBI:64198': 8,
  'MESH:D019164': 18,
  'MESH:C059416': 3,
  'CHEBI:CHEBI:15710': 2},
 'f1': {'mean': 0.993068, 'std': 0.002061},
 'precision': {'mean': 0.987107, 'std': 0.004015},
 'recall': {'mean': 0.999257, 'std': 0.001487},
 'CHEBI:CHEBI:15710': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'ungrounded': {'f1': {'mean': 0.67, 'std': 0.345832},
  'pr': {'mean': 0.616667, 'std': 0.331662},
  'rc': {'mean': 0.75, 'std': 0.387298}},
 'HGNC:1848': {'f1': {'mean': 0.978069, 'std': 0.031578},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 0.958889, 'std': 0.058542}},
 'MESH:D019164': {'f1': {'mean': 0.914286, 'std': 0.069985},
  'pr': {'mean': 0.85, 'std': 0.122474},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'MESH:D018893': {'f1': {'mean': 0.994077, 'std': 0.001757},
  'pr': {'mean': 0.999206, 'std': 0.001587},
  'rc': {'mean': 0.98900

In [21]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [22]:
disamb.dump(model_name, results_path)

In [24]:
print(disamb.info())

Disambiguation model for BAL

Produces the disambiguations:
	Bronchoalveolar Lavage*	MESH:D018893
	CEL*	HGNC:1848
	CEL	CEL
	Liver, Artificial	MESH:D019164
	benzaldehyde lyase	MESH:C059416
	betaine aldehyde	CHEBI:CHEBI:15710
	dimercaprol	CHEBI:CHEBI:64198

Class level metrics:
--------------------
Grounding             	Count	F1     
Bronchoalveolar Lavage*	1259	0.99408
                   CEL*	  86	0.97807
     Liver, Artificial	  18	0.91429
            Ungrounded	  17	   0.67
           dimercaprol	   8	0.46667
    benzaldehyde lyase	   3	    0.2
      betaine aldehyde	   2	    0.0

Weighted Metrics:
-----------------
	F1 score:	0.99307
	Precision:	0.98711
	Recall:		0.99926

* Positive labels
See Docstring for explanation



In [25]:
model_to_s3(disamb)